In [ ]:
import os
save_directory= "Model"
train_address = "Train"
test_address = "Test"

if not os.path.exists(test_address):
  os.makedirs(test_address)
  !wget -q https://raw.githubusercontent.com/circulosmeos/gdown.pl/master/gdown.pl -O gdown.pl
  !pip install efficientnet
  !perl gdown.pl "https://drive.google.com/u/0/uc?export=download&confirm=n51E&id=1rxlpshNKP2gQ6NP9Wpvq-jUdbk5t6-Ub" "Red-Cards.zip"
  !unzip Red-Cards.zip
  !mv Red-Cards Test/Red-Cards
  !perl gdown.pl "https://drive.google.com/u/0/uc?export=download&confirm=Ez4P&id=1tvWzG3stQaY6fqs8O6F_3Ec_qguZU3mC" "Yellow-Cards.zip"
  !unzip Yellow-Cards.zip
  !mv Yellow-Cards Test/Yellow-Cards

if not os.path.exists(train_address):
  os.makedirs(train_address)
  !perl gdown.pl "https://drive.google.com/u/0/uc?export=download&confirm=nJ31&id=1sQuc6JoE5CoKq6JYe6i75AoWEBngAVb8" "Red-Cards.zip"
  !unzip Red-Cards.zip
  !mv Red-Cards Train/Red-Cards
  !perl gdown.pl "https://drive.google.com/u/0/uc?export=download&confirm=vrjS&id=1IozEkL1xhRoG9oqnBvdozrlTcTsqk9s6" "Yellow-Cards.zip"
  !unzip Yellow-Cards.zip
  !mv Yellow-Cards Train/Yellow-Cards


In [4]:
!pip install efficientnet
import efficientnet.keras as enet
import keras
from keras.callbacks import EarlyStopping
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Multiply, add
from keras.layers import GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.regularizers import l2
import matplotlib.image as mpimg
from keras.layers import Input, Reshape, Dense, Lambda, Activation
from keras.layers import Input, Reshape, Dense, Lambda, Activation
from keras.optimizers import Adam, RMSprop, SGD
from keras.models import Model
from keras.regularizers import l2
from keras.applications.vgg16 import VGG16
from keras.initializers import *
import numpy as np
import keras.backend as K
import pathlib
from PIL import Image
import cv2
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model
import pandas as pd
import os
from keras.optimizers import Adam,RMSprop, SGD , Nadam, Adadelta
def _outer_product(x):

    return K.batch_dot(x[0], x[1], axes=[1, 1]) / x[0].get_shape().as_list()[1]

def _signed_sqrt(x):

    return K.sign(x) * K.sqrt(K.abs(x) + 1e-9)

def _l2_normalize(x, axis=-1):

    return K.l2_normalize(x, axis=axis)
def buil_bcnn(
        all_trainable=False,

        size_height=448,
        size_width=448,
        no_class=200,
        no_last_layer_backbone=17,

        name_optimizer='sgd',
        learning_rate=1.0,
        decay_learning_rate=0.0,
        decay_weight_rate=0.0,

        name_initializer='glorot_normal',
        name_activation='softmax',
        name_loss='categorical_crossentropy'
    ):

    input_tensor = Input(shape=[size_height, size_width, 3])
    pre_train_model = enet.EfficientNetB0(weights = "imagenet", include_top=False, input_tensor=input_tensor)


    for layer in pre_train_model.layers:
        layer.trainable = all_trainable



    model_detector = pre_train_model
    output_detector = model_detector.layers[no_last_layer_backbone].output
    shape_detector = model_detector.layers[no_last_layer_backbone].output_shape

    model_extractor = pre_train_model
    output_extractor = model_extractor.layers[no_last_layer_backbone].output
    shape_extractor = model_extractor.layers[no_last_layer_backbone].output_shape

    output_detector = Reshape(
        [shape_detector[1]*shape_detector[2], shape_detector[-1]])(output_detector)
    output_extractor = Reshape(
        [shape_extractor[1]*shape_extractor[2], shape_extractor[-1]])(output_extractor)

    x = Lambda(_outer_product)([output_detector, output_extractor])
    x = Reshape([shape_detector[-1]*shape_extractor[-1]])(x)
    x = Lambda(_signed_sqrt)(x)
    x = Lambda(_l2_normalize)(x)



    if name_initializer is not None:
        name_initializer = eval(name_initializer+'()')

    x = Dense(
        units=no_class,
        kernel_initializer=name_initializer,
        kernel_regularizer=l2(decay_weight_rate))(x)
    output_tensor = Activation(name_activation)(x)
    model_bcnn = Model(inputs=[input_tensor], outputs=[output_tensor])

    if name_optimizer == 'adam':
        optimizer = adam(lr=learning_rate, decay=decay_learning_rate)
    elif name_optimizer == 'rmsprop':
        optimizer = RMSprop(lr=learning_rate, decay=decay_learning_rate)
    elif name_optimizer == 'sgd':
        optimizer = SGD(lr=learning_rate, decay=decay_learning_rate, momentum=0.9, nesterov=None)
    else:
        raise RuntimeError('Optimizer should be one of Adam, RMSprop and SGD.')

    model_bcnn.compile(loss=name_loss, optimizer=optimizer, metrics=['accuracy'])


    return model_bcnn


In [ ]:

BATCH_SIZE = 16
test_nb = 1000
train_nb = 11000
num_classes = 2
img_size= 224
classes = []

train_path = "Train"
test_path = "Test"



train_datagen = ImageDataGenerator(rescale = 1.0/255, 
                                   zoom_range=[1.0,2.0],
                                   rotation_range=90,
                                   horizontal_flip=True,
                                   vertical_flip=True)
test_datagen = ImageDataGenerator(rescale = 1.0/255)

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_size, img_size),
        batch_size=BATCH_SIZE,
        seed = 13)

validation_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(img_size, img_size),
        batch_size=BATCH_SIZE,
        seed = 13)

model = buil_bcnn(
    size_height=224,
    size_width=224,
    no_class=2)

model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=['accuracy'])


history = model.fit_generator(train_generator,
                    steps_per_epoch=train_nb/ 16 ,
                    epochs=60,
                    validation_data=validation_generator,
                    validation_steps=int(test_nb/16),
                    verbose=1)  #, checkpointer

import datetime
now = datetime.datetime.now()
model.evaluate(validation_generator)


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig("model accuracy")
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig("model loss")

print("[INFO] serializing network...")
model.save("model_saved")

